# sPOD-NN for synthetic data

In [ ]:
import sys
sys.path.append('../sPOD/lib/')
sys.path.append('../DL-ROM/LIB/')

In [ ]:
from synthetic_sup import synthetic_sup
import numpy as np

## Data generation / sPOD of the data

In [ ]:
df = synthetic_sup(spod_iter=300, plot_offline_data=True)

## Input data for sPOD-NN 

In [ ]:
# We collect the time amplitudes, shifts and the parameters for the training as well as the testing data.
TA_TRAIN = df.TA_TRAIN
SHIFTS_TRAIN = df.SHIFTS_TRAIN
PARAMS_TRAIN = df.PARAMS_TRAIN
TA_TEST = df.TA_TEST
SHIFTS_TEST = df.SHIFTS_TEST
PARAMS_TEST = df.PARAMS_TEST
TA_POD_TRAIN = df.TA_POD_TRAIN
TA_POD_TEST = df.TA_POD_TEST

## ---------------------------------------------------

In [ ]:
# ###################################### Only for DoF study #######################################
# # No need to be performed again 
# # The results are shown in the paper

# import matplotlib.pyplot as plt
# import os
# from sklearn.utils.extmath import randomized_svd
# from Helper import *
# ############################################

# frame_wise_sPOD = [1, 1]
# time_amplitudes_1 = TA_TRAIN[:df.nmodes, :]
# time_amplitudes_2 = TA_TRAIN[df.nmodes:, :]
# frame_amplitudes_training = [
#     time_amplitudes_1[:frame_wise_sPOD[0], :],
#     time_amplitudes_2[:frame_wise_sPOD[1], :]
# ]
# TA_TRAIN = np.concatenate(frame_amplitudes_training, axis=0)

# df.U_list[0] = df.U_list[0][:, :frame_wise_sPOD[0]]
# df.U_list[1] = df.U_list[1][:, :frame_wise_sPOD[1]]
# spodModes = frame_wise_sPOD

# time_amplitudes_1_test = df.U_list[0].transpose() @ df.q1_test
# time_amplitudes_2_test = df.U_list[1].transpose() @ df.q2_test
# amplitudes_test = np.concatenate((time_amplitudes_1_test, time_amplitudes_2_test), axis=0)
# df.TA_TEST = amplitudes_test
# TA_TEST = df.TA_TEST

# ############################################   

# frame_amplitudes_list_interp = []
# for frame in range(2):
#     Nmodes = spodModes[frame]
#     VT = frame_amplitudes_training[frame]
#     amplitudes = [np.reshape(VT[n, :], [df.Nsamples_train, len(df.t)]).T for n in range(Nmodes)]
#     frame_amplitudes_list_interp.append(amplitudes)

# df.TA_list_interp = frame_amplitudes_list_interp      
# ############################################    

# frame_wise_POD = sum(frame_wise_sPOD) + 2
# df.U_POD_TRAIN = df.U_POD_TRAIN[:, :frame_wise_POD]
# df.TA_POD_TRAIN = df.TA_POD_TRAIN[:frame_wise_POD, :]
# TA_POD_TRAIN = df.TA_POD_TRAIN

# df.TA_POD_TEST = df.U_POD_TRAIN.transpose() @ df.q_test
# TA_POD_TEST = df.TA_POD_TEST
# ############################################

# data_shape = [len(df.x), 1, 1, 5*len(df.t)]
# dx = df.x[1] - df.x[0]
# L = [df.x[-1]]

# q_train = [df.U_list[0] @ frame_amplitudes_training[0], 
#           df.U_list[1] @ frame_amplitudes_training[1]]

# trafos = df.trafos_train

# NumFrames = 2
# q_sPOD = 0
# for frame in range(NumFrames):
#     q_sPOD += trafos[frame].apply(q_train[frame])
# ############################################
# q_POD = df.U_POD_TRAIN @ df.TA_POD_TRAIN
# ############################################
# q_original = df.q_train[0:len(df.x), :]
# ############################################

# num1 = np.sqrt(np.mean(np.linalg.norm(q_original - q_sPOD, 2, axis=1) ** 2))
# den1 = np.sqrt(np.mean(np.linalg.norm(q_original, 2, axis=1) ** 2))

# num2 = np.sqrt(np.mean(np.linalg.norm(q_original - q_POD, 2, axis=1) ** 2))
# den2 = np.sqrt(np.mean(np.linalg.norm(q_original, 2, axis=1) ** 2))

# df.D = 1
# df.nmodes = 1

# print("Error for sPOD recons. is {}".format(num1 / den1))
# print("Error for POD recons. is {}".format(num2 / den2))


## ---------------------------------------------------

In [ ]:
print("Grid, Nx : {}, Nt : {}".format(df.Nx, df.Nt))
print("Number of sPOD frames : {}".format(df.NumFrames))
print("Number of modes per frame : {}".format(df.nmodes))
print("Number of parameter instances : {}".format(int(int(TA_TRAIN.shape[1]) / df.Nt)))
print("Size of training matrix : {} x {}".format(int(TA_TRAIN.shape[0]), int(TA_TRAIN.shape[1])))

In [ ]:
# Data manipulations needed for the network input
shifts_train = np.concatenate((np.reshape(SHIFTS_TRAIN[0], newshape=[1, -1]), np.reshape(SHIFTS_TRAIN[1], newshape=[1, -1])), axis=0)
shifts_test = np.concatenate((np.reshape(SHIFTS_TEST[0], newshape=[1, -1]), np.reshape(SHIFTS_TEST[1], newshape=[1, -1])), axis=0)

ta_train = np.concatenate((TA_TRAIN, shifts_train), axis=0)
ta_test = np.concatenate((TA_TEST, shifts_test), axis=0)

## Network prediction

In [ ]:
params_sPOD = {
        'scaling': True,  # true if the data should be scaled
        'full_order_model_dimension': df.Nx,  # N_h
        'reduced_order_model_dimension': ta_train.shape[0],  # N
        'totalModes': ta_train.shape[0] - df.NumFrames,  # Total number of modes for all the frames
        'num_early_stop': 100000  # early stop criteria 
    
    }
params_POD = {
        'scaling': True,  # true if the data should be scaled
        'full_order_model_dimension': df.Nx,  # N_h
        'reduced_order_model_dimension': TA_POD_TRAIN.shape[0],  # N
        'totalModes': TA_POD_TRAIN.shape[0],  # Total number of modes for all the frames
        'num_early_stop': 500  # early stop criteria 
    }

In [ ]:
# training the model
from DFNN import run_model 
print("#################################")
print("sPOD-NN")
trained_model_sPOD, _, scaling_sPOD = run_model(ta_train, PARAMS_TRAIN, epochs=150000, lr=0.0025, loss_type='L1', 
                                                logs_folder='./DNN_result/synthetic/training_results_sPOD', 
                                                params=params_sPOD, batch_size=50)
print("#################################\n")
print("#################################")
print("POD-NN")
trained_model_POD, _, scaling_POD = run_model(TA_POD_TRAIN, PARAMS_TRAIN, epochs=150000, lr=0.0025, loss_type='L1',
                                              logs_folder='./DNN_result/synthetic/training_results_POD', 
                                              params=params_POD, batch_size=10)
print("#################################\n")

In [ ]:
# loading the model
import torch
import pathlib
import os

log_folder_base_sPOD = 'DNN_result/synthetic/training_results_sPOD/'
log_folder_trained_model_sPOD = sorted(pathlib.Path(log_folder_base_sPOD).glob('*/'), key=os.path.getmtime)[-1]
PATH_sPOD = str(log_folder_trained_model_sPOD) + '/trained_weights/' + 'weights.pt'

log_folder_base_POD = 'DNN_result/synthetic/training_results_POD/'
log_folder_trained_model_POD = sorted(pathlib.Path(log_folder_base_POD).glob('*/'), key=os.path.getmtime)[-1]
PATH_POD = str(log_folder_trained_model_POD) + '/trained_weights/' + 'weights.pt'

PATH_sPOD = 'DNN_result/synthetic/training_results_sPOD/2023_02_10__15-10-53/trained_weights/weights.pt'
PATH_POD = 'DNN_result/synthetic/training_results_POD/2023_02_10__15-20-51/trained_weights/weights.pt'

In [ ]:
from DFNN import scale_params
PARAMS_TEST_sPOD = scale_params(PARAMS_TEST, params_sPOD, scaling_sPOD)
PARAMS_TEST_POD = scale_params(PARAMS_TEST, params_POD, scaling_POD)

In [ ]:
# testing the model
from DFNN import test_model 
rel_err_sPOD, results_predicted_sPOD = test_model(ta_test, PARAMS_TEST_sPOD, saved_model=True, 
                                                  PATH_TO_WEIGHTS=PATH_sPOD, params=params_sPOD,
                                                  scaling=scaling_sPOD, batch_size=50) 
rel_err_POD, results_predicted_POD = test_model(TA_POD_TEST, PARAMS_TEST_POD, saved_model=True,
                                               PATH_TO_WEIGHTS=PATH_POD, params=params_POD,
                                               scaling=scaling_POD, batch_size=10)
print(rel_err_sPOD, rel_err_POD)

## Online prediction analysis

In [ ]:
# This cell is reserved for data manipulations for the online analysis
TA_sPOD_pred = results_predicted_sPOD[:-2, :]
shifts_sPOD_pred = results_predicted_sPOD[-2:, :]
TA_POD_pred = results_predicted_POD

In [ ]:
errors = df.OnlinePredictionAnalysis(TA_sPOD_pred, shifts_sPOD_pred, TA_POD_pred, plot_online_data=True)

## Error plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from Helper import save_fig
from statistics import mean

impath = "../plots/images_synthetic/"
os.makedirs(impath, exist_ok=True) 

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern"]})

SMALL_SIZE = 16   # 16
MEDIUM_SIZE = 18   # 18
BIGGER_SIZE = 20   # 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
truncated_modes = np.array([2, 6, 10, 14, 16])
E_sPOD = np.array([0.26982, 0.02641, 0.00499, 0.00222, 0.00073])
E_POD = np.array([0.78781, 0.63762, 0.51599, 0.40523, 0.35954])
E_sPOD_NN = np.array([0.31729, 0.12410, 0.09889, 0.09411, 0.06559])
E_sPOD_I = np.array([0.26982, 0.02641, 0.00501, 0.00223, 0.00079])
E_POD_NN = np.array([0.9837, 0.9833, 0.9822, 1.0, 1.0])

err = errors[0]
err_min = [min(x) for x in err]
err_max = [max(x) for x in err]
err_mean = [mean(x) for x in err]

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

axs[0].semilogy(truncated_modes, E_sPOD, color="peru", linestyle='-', marker=".", label=r"$E^{\mathrm{sPOD}}$")
axs[0].semilogy(truncated_modes, E_POD, color="brown", linestyle='-', marker=".", label=r"$E^{\mathrm{POD}}$")
axs[0].semilogy(truncated_modes, E_sPOD_NN, color="red", linestyle='--', marker="*", label=r"$E^{\mathrm{sPOD-NN}}_{\mathrm{tot}}$")
axs[0].semilogy(truncated_modes, E_sPOD_I, color="blue", linestyle='--', marker="*", label=r"$E^{\mathrm{sPOD-I}}_{\mathrm{tot}}$")
axs[0].semilogy(truncated_modes, E_POD_NN, color="black", linestyle='--', marker="*", label=r"$E^{\mathrm{POD-NN}}_{\mathrm{tot}}$")
axs[0].set_xlabel('Number of modes')
axs[0].set_ylabel('Errors')
axs[0].grid()
axs[0].legend(loc='lower left')

axs[1].semilogy(df.t, err_max, color="teal", linestyle='--', label=r"max$(E^{\mathrm{sPOD-NN}}_{\mathrm{t}})$")
axs[1].semilogy(df.t, err_mean, color="orange", linestyle='--', label=r"mean$(E^{\mathrm{sPOD-NN}}_{\mathrm{t}})$")
axs[1].semilogy(df.t, err_min, color="dimgrey", linestyle='--', label=r"min$(E^{\mathrm{sPOD-NN}}_{\mathrm{t}})$")
axs[1].set_xlabel(r"time $t$")
axs[1].grid()
axs[1].legend(loc='lower right')


save_fig(filepath=impath + 'Rel_err', figure=fig)
fig.savefig(impath + "Rel_err" + ".eps", format='eps',dpi=600, transparent=True)